# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

In [1]:
# importeren libs
import numpy as np
import pandas as pd
# import datetime
from itertools import product
# import itertools
import pickle
import copy


# import sys
# import dovpanda
import Importeer 
import Hulpfuncties
import CrewCombined
import Dataframe
import Controleren
from Class import MogelijkeCombinaties, Rooster

In [2]:
# print(sys.getsizeof(dfSprint))

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

Het gegeven excelbestand wat de informatie vat, is op een aantal punten aangepast voordat het is geimporteerd. 
Het bestand heeft voor iedere skill een aparte kolom. Mocht de skill nodig zijn voor de gegeven opdracht, dan staat er een 'x' in deze kolom. Ditzelfde is gedaan met de skills van medewerkers, de dagen waarop de medewerkers werken en voor de kamers waarin de opdrachten moeten worden uitgevoerd. Voor de additional tasks zijn er een aantal opdracht die in iedere sprint moeten worden uitgevoerd. Deze zijn opgesplitst, voor iedere sprint een aparte lijn. Hierbij is ook een kolom toegevoegd met het aantal frequenties voor deze opdracht. Ook is er een kolom toegevoegd bij het tabblad 'additional tasks' waarin staat aangegeven met hoeveel personen het minimaal moet worden uitgevoerd.
Hieronder worden de verschillende tabbladen ingelezen en toegekent aan een dataframe.
Verdere bewerkingen op de data worden gedaan in dit document.


In [3]:
# CONSTANTE
SPRINT = 'S1'
OSHA = [4,9,0]
crewDirecter = 6
science = 9
PR = 0
legal = 4
prepCheck = 0
prepTrouble = 3

# ik weet niet hoe vaak deze nog wordt gebruikt.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

In [4]:
# Importeren van verschillende sheets. Dit wordt gedaan met functies.
# in deze functies wordt het ook in het goede format gezet.

dfCrew = Importeer.Crew()
dfRooms = Importeer.Rooms()
dfSprint = Importeer.dfSprint(SPRINT)

In [5]:
# CrewCombined.MakeCrewCombined(dfCrew) # voor het maken van dfCrewcombined.csv (duurt even)
dfCrewCombined = pd.read_csv('CrewCombined.csv') # lezen van csv zodra die in wdir staat.

**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken, daarna een half uur pauze
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' kan voor ieder crewlid worden gedaan, geen beperking in aantal 
- crewleden moet te samen aan de skills komen voor de opdracht.
- Mensen die skill één niet hebben, hoeven deze niet te leren
- Een crewleden kunnen alleen aan eengesloten uren aan een project task werken. Dit mag niet worden doorbroken met een andere task. Met additional tasks mag dit wel. 
- iedereen moet tegelijk aan het project werken als ze op hetzelfde project staan.

Moet A volledig worden afgerond of maar 1 keer van het aantal frequenties? Moet volledig worden ingepland

We maken een soort kalender, startend op 6 januari 2020 en geen weekend dagen erin.
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- een kolom met dataframes met de volgende punten:
    - tijd per halfuur
    - persoon
    - aantal uren te gaan
    - hierin staan de projecten met de kamer (is dit zo?)
- Of het een werkdag is (T/F)
- In welke sprint de dag zich bevindt.

In [6]:
dfKalenderCrew = Dataframe.LegeKalender()

In [7]:
dfKalenderRoom = pickle.loads(pickle.dumps(dfKalenderCrew)) # het maken van een kalender van de kamers, tot dit punt waren de df's hetzelfde

In [8]:
# dfSprint.iloc[:,26]= dfSprint.apply(lambda rij : Hulpfuncties.ToevoegenMogelijkeCombinaties(rij, 
#                                                     dfSprint, dfCrewCombined), axis = 1)

# dfSprint.iloc[:,27] = dfSprint['Frequency']*dfSprint['MinReqCrew']

In [9]:
# # het schrijven van dfSprint naar een pickle bestand
# filehandler = open('dfSprint1_begin.obj', 'wb') 
# pickle.dump(dfSprint, filehandler)

In [10]:
# het lezen van dfSprint, met meerdere keren runnen een heel stuk sneller
filehandler = open('dfSprint1_begin.obj', 'rb') 
dfSprint = pickle.load(filehandler)

In [11]:
dfSprint = dfSprint.sort_values(['AantalMogelijkheden'], ascending= [False])
dfSprint = dfSprint.reset_index(drop=True)

In [12]:
Eisen = dfSprint.Eis.unique().tolist()
Eisen.remove('no')

dfSprint = (dfSprint.loc[dfSprint['Task'].isin(Eisen)].append(dfSprint.loc[~dfSprint['Task'].isin(Eisen)])).reset_index(drop=True)

In [13]:
StandaardRooster = Dataframe.RoosterCrew(dfCrew)
dfRoosterKamer = Dataframe.RoosterKamer(dfRooms)

In [14]:
dfCrew['Uren']=dfCrew.apply(lambda row: (row.mon + row.tue + row.wed + row.thu + row.fri)*70, axis=1)

In [15]:
# inplannen stand-up op dinsdagochtend
dfRoosterDinsdag = copy.deepcopy(StandaardRooster)
dfRoosterDinsdag.iloc[1:10,1:14] = "weekly stand-up"
dfRoosterDinsdag.iloc[0,1:14] = 2.5

dfCrew['Uren'] -= 45

dfRoosterDinsdag.iloc[10:15,[x+1 for x in OSHA]] = "OSHA"
dfRoosterDinsdag.iloc[0,[x+1 for x in OSHA]] = 0

dfRoosterKamerDinsdag = copy.deepcopy(dfRoosterKamer)
dfRoosterKamerDinsdag.loc[0:5, 'workshop A'] ="weekly stand-up"

In [16]:
for i in range(0,5): # het maken van de lege roosters voor week 1
    if i== 1:
        rooster = copy.deepcopy(dfRoosterDinsdag)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamerDinsdag))
    else:
        rooster = copy.deepcopy(StandaardRooster)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamer))
    
for i in range(1,10): # het kopieren van de roosters van week 1 voor weken in de sprint
    for j in range(0,5):
        dfKalenderCrew.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderCrew.DagRooster[j])
        dfKalenderRoom.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderRoom.DagRooster[j])

C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


!TODO opdrachten over twee dagen

In [17]:
dfSprintAddprep, dfSprintAddpost = Importeer.AdditionalTasks(SPRINT)

In [18]:
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name='Additional tasks')
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfAdditionalTasks = dfAdditionalTasks[dfAdditionalTasks['Task'] != 0].reset_index(drop=True)
dfAdditionalTasks['aantal keer'].replace(0, 1, inplace=True)
dfSprintAdd = pd.DataFrame((dfAdditionalTasks.loc[~dfAdditionalTasks['Task'].str.contains(SPRINT)]) ).reset_index(drop=True)
dfSprintAdd = pd.DataFrame((dfSprintAdd[dfSprintAdd['Sprint']==1]) ).reset_index(drop=True)
dfSprintAdd['Voltooid'] = False
dfSprintAdd.loc[:, 'Crew'] = [20] * len(dfSprintAdd)
dfSprintAdd['Crew'] = dfSprintAdd['Crew'].astype(object)

In [19]:
def InplannenOSHA(urenPerWeek, dfKalenderCrew, dfAdd):
    duur = dfAdd.loc[dfAdd['Task'].str.contains("OSHA"), 'DurationExp.1'] + 3*dfAdd.loc[dfAdd['Task'].str.contains("OSHA"), 'DurationStd.1'] - 25 
    duur = duur.tolist()[0]

    for crew in OSHA:
        dfCrew['Uren'].iloc[crew] -= (duur + 25)
    # voor sprint 1 en 2
    weken = [1,9,10]
    for week in weken:
        for crew in OSHA:
            dfKalenderCrew.iloc[(week-1)*5+3,:].DagRooster.dfRooster.iloc[1:(urenPerWeek*2+2),crew+1] = "OSHA"
            dfKalenderCrew.iloc[(week-1)*5+3,:].DagRooster.dfRooster.iloc[0,crew+1] = dfKalenderCrew.iloc[(week-1)*5+3,:].DagRooster.dfRooster.iloc[0,crew+1] - urenPerWeek
        duur -= urenPerWeek
        if (urenPerWeek > duur):
            urenPerWeek = duur
    
    
#     task = dfAdd.loc[dfAdd['Task'].str.contains("OSHA"), 'Task']
    dfAdd.loc[dfAdd['Task'].str.contains("OSHA"), 'Voltooid'] = True
    dfAdd.at[dfAdd.index[dfAdd['Task'].str.contains("OSHA")].tolist()[0], 'Crew'] = OSHA
    
    return [dfKalenderCrew, dfAdd]


In [20]:
dfKalenderCrew, dfSprintAdd = InplannenOSHA(5, dfKalenderCrew, dfSprintAdd)

C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [21]:
# reserveren uren director etc.

# STD moet hier nog wel bij 
dfCrew.Uren.iloc[crewDirecter] -= dfSprintAdd[dfSprintAdd.Task == "crew director"]["DurationExp.1"].tolist()[0]
dfCrew.Uren.iloc[crewDirecter] -= (dfSprintAdd[dfSprintAdd.Task == "meetup with stakeholders"]["DurationExp.1"].tolist()[0]+3*dfSprintAdd[dfSprintAdd.Task == "meetup with stakeholders"]["DurationStd.1"].tolist()[0])
# Werkt, alleen pakt hij standaard de verkeerd (met 12+3*3)

dfCrew.Uren.iloc[science] -= (dfSprintAdd[dfSprintAdd.Task == "scientific advisor"]["DurationExp.1"].tolist()[0]+3*dfSprintAdd[dfSprintAdd.Task == "scientific advisor"]["DurationStd.1"].tolist()[0])

dfCrew.Uren.iloc[PR] -= (dfSprintAdd[dfSprintAdd.Task == "PR"]["DurationExp.1"].tolist()[0]+3*dfSprintAdd[dfSprintAdd.Task == "PR"]["DurationStd.1"].tolist()[0])
dfCrew.Uren.iloc[legal] -= (dfSprintAdd[dfSprintAdd.Task == "legal assistant"]["DurationExp.1"].tolist()[0]+3*dfSprintAdd[dfSprintAdd.Task == "legal assistant"]["DurationStd.1"].tolist()[0])
dfCrew.Uren.iloc[prepCheck] -= (dfSprintAdd[dfSprintAdd.Task == "prep check-up"]["DurationExp.1"].tolist()[0]+3*dfSprintAdd[dfSprintAdd.Task == "prep check-up"]["DurationExp.1"].tolist()[0])
dfCrew.Uren.iloc[prepTrouble] -= (dfSprintAdd[dfSprintAdd.Task == "prep troubleshooting"]["DurationExp.1"].tolist()[0]+3*dfSprintAdd[dfSprintAdd.Task == "prep troubleshooting"]["DurationExp.1"].tolist()[0])


In [22]:
def CheckCrewUren(crewlijst, dfCrew, duration):
    
    tempCrew = pickle.loads(pickle.dumps(dfCrew))
    for crew in crewlijst:
        if crew == 20:
            return [True, dfCrew]
        elif (dfCrew['Uren'].iloc[crew] >= duration):
            dfCrew['Uren'].iloc[crew] -= duration
        else:
            return [False, tempCrew]
    return [True, dfCrew]

In [23]:
crewlijst = Hulpfuncties.somOverigeUren([13,14,15], [1], dfKalenderCrew)

In [24]:
taskdf = dfSprintAddprep.iloc[0,:]

In [25]:
taskdf

Task                 prep S1C1T1A
ReqSkill 1                      0
ReqSkill 2                      0
ReqSkill 3                      0
ReqSkill 4                      0
ReqSkill 5                      0
ReqSkill 6                      0
ReqSkill 7                      0
ReqSkill 8                      0
ReqSkill 9                      0
ReqSkill 10                     0
ReqSkill 11                     0
ReqSkill 12                     0
ReqSkill 13                     0
ReqRoom technical               0
ReqRoom **                      0
ReqRoom ***                     0
ReqRoom ****                    0
ReqRoom 4                       1
ReqRoom 5                       1
AdditionalReq                   0
DurationExp.1                   7
DurationStd.1                   1
DurationExp.2                   7
DurationStd.2                   1
DurationExp.3                   7
DurationStd.3                   1
People.1                        1
People.2                        1
People.3      

In [26]:
crewList = crewlijst['crew'].tolist()

for crew in crewList: # checken voor het totaal aantal uren en extra 
    if ((dfCrew['Uren'].iloc[crew] < taskdf["DurationExp.1"]) | (crew >12)):
        crewlijst = crewlijst[crewlijst.crew != crew]
if (len(crewlijst) < taskdf["People.1"]):
    print(False)

False


In [27]:
crewlijst

,crew,som


In [28]:
def InplannenAdd(taskdf, dfKalenderCrew, dfKalenderRoom, dfSprint, dfAdd, dfCrew, prep = True, inplannen = False): 

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempAdd = pickle.loads(pickle.dumps(dfAdd))
    dfCopyCrew = pickle.loads(pickle.dumps(dfCrew))
    
    gelukt = False

    if prep:
        dagIndex = 0
        weken = [1]
        maxDagIndex = 5
    else:
        dagIndex = 40
        weken = [9,10]
        maxDagIndex = 50
    # crew zelf maken
    crew = []

    if (taskdf.AdditionalReq == 0):
        
        task = taskdf.Task[5:]
        crewlijst = dfSprint[dfSprint.Task == task].iloc[0,:].Crew
        
        minCrew = taskdf['People.1']

        crewlijst = Hulpfuncties.somOverigeUren(crewlijst, weken, dfKalenderCrew)
        crewList = crewlijst['crew'].tolist()
        
        
        # iets returnen zodra er geen crew meer is.
        for crew in crewList: # checken voor het totaal aantal uren en extra 
            if ((dfCrew['Uren'].iloc[crew] < taskdf["DurationExp.1"])):
                crewlijst = crewlijst[crewlijst.crew != crew]
#         if (len(crewlijst) < minCrew):
#             return [False, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]
        
        print("de crewList: " + str(crewlijst))
        crewlijst = crewlijst['crew'].iloc[:minCrew].to_list()
        crewlijst.extend([20]*(6-minCrew))
       
    elif ((len(taskdf.AdditionalReq) > 5) & (len(taskdf.AdditionalReq) < 8)):
        try:
            crewlijst = dfSprint[dfSprint.Task == taskdf.AdditionalReq].iloc[0,:].Crew
        except:
            filehandler = open('dfSprint3_eind.obj', 'rb') # Moet worden verandert met de sprint
            dfSprintVorig = pickle.load(filehandler)
            crewlijst = dfSprintVorig[dfSprintVorig.Task == taskdf.AdditionalReq].iloc[0,:].Crew
            
    elif (len(taskdf.AdditionalReq) > 8 ):
        taskA = taskdf.AdditionalReq[0:7]
        taskB = taskdf.AdditionalReq[-7:]
        taskACrew = dfSprint[dfSprint.Task == taskA].iloc[0,:].Crew
        taskBCrew = dfSprint[dfSprint.Task == taskB].iloc[0,:].Crew
        crewlijst = [20]*6
        b = list(set(taskACrew)&set(taskBCrew))
        try:
            b.remove(20)
        except:
            None
        crewlijst[0] = b[0]
        
    a = list(set(crewlijst))
    try:
        a.remove(20)
    except:
        None
    aantalCrew = len(a)
    kolom = 1
    if (int(taskdf['People.1'])==int(aantalCrew)):
        kolom = 1
    elif (int(taskdf['People.2'])==int(aantalCrew)):
        kolom = 2
    elif (int(taskdf['People.3'])==int(aantalCrew)):
        kolom = 3
    
 # tot en met hier klopt de functie
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    
    if (int(round(taskdf[str("DurationExp." + str(kolom))]*2)) == 0):
        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
        gelukt = True
    elif (len(crewlijst) < taskdf['People.1']):
        gelukt = False
    else:
        
            
        while halfuur < int(round(taskdf[str("DurationExp." + str(kolom))]*2)):
            for i in crewlijst:
                if (i == 20):
                    break
#                 print("de controle is: " + str(((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5))))
                if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)):
                    controle = False
                    break 
                lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment

            if ((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle):
                rijbegin , rijeind = Hulpfuncties.contains([0], lijstVrijeUren)
                
                kamerGevonden = False

                kamers = Hulpfuncties.ZoekKamers(taskdf.iloc[14:20], dfAdd)

                ## hier nog even naar kijken of het klopt dat er niet naar rijeind en begin wordt gekeken !TODO
                for kamer in kamers:
                    
                    if (dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1].tolist() == [0]):
                        
                        if inplannen:
                            dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1] = taskdf.Task
                        kamerGevonden = True
                        controle = True
                        break
                    elif ((kamer == kamers[-1]) & (kamerGevonden == False)):                       
                        controle = False

                if (controle == True):
                    for i in crewlijst:
                        if i == 20:
                            break
                        
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task 
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5)

                    dfAdd.at[dfAdd.index[dfAdd['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                    halfuur +=1

                    if (halfuur == (int(round(taskdf[str("DurationExp." + str(kolom))]*2)))):

                        geluktCrew, dfCrew = CheckCrewUren(crewlijst, dfCrew, (int(round(taskdf[str("DurationExp." + str(kolom))]))))
                        if not geluktCrew:
                            print("er is iets fout gegaan")
                        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
                        print(str(taskdf.Task) + "is daadwerkelijk ingepland")

                        if inplannen:
                            return [True, dfKalenderCrew, dfKalenderRoom, dfAdd, dfCrew]
                        else:
                            return [True, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]

            else:

                dagIndexKopie+=1

                controle = True
                lijstVrijeUren = [0]*16
                if (dagIndexKopie == maxDagIndex):
                    break

    print(str(taskdf.Task) + " Niet ingepland of: " + str(gelukt))
    return [gelukt, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]

In [29]:
def get_last_non_zero_index(d, default=16):
    rev = (len(d) - idx for idx, item in enumerate(reversed(d), 1) if item)
    return next(rev, default)

In [30]:
def index_of_first_nonzero(lst, duration, default= False):
    for i, value in enumerate(lst):
        if value != 0:
            return i >= duration
    return True if (sum(lst)== 0) else default

In [31]:
# %%pixie_debugger

def InplannenOpdracht(taskdf, dagIndex, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfAddPost, dfSprint, dfCrew, crewA = []): 
# task moet een rij van het dataframe dfSprint zijn en een dag (als index) van een rij in dfKalenderCrew (verschil ten opzichten van 2020-1-6)
        
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempSprint = pickle.loads(pickle.dumps(dfSprint))
    dfTempPrep = pickle.loads(pickle.dumps(dfAddPrep))
    dfTempPost = pickle.loads(pickle.dumps(dfAddPost))
    dfCopyCrew = pickle.loads(pickle.dumps(dfCrew))
#     tabulist=[]               #################
    for crewlijst in taskdf.Mogelijkheden.dfMogelijkheden.iloc[:,1].tolist():
#         checkTabu=True             #################
#         for i in tabulist:    #################
#             if i in crewlijst:#################
#                 checkTabu=False   #################
#         if checkTabu:            ################
#             tabulist.append(crewlijst) #################
        checkUren, dfCrew = CheckCrewUren(crewlijst, dfCrew, taskdf.Duration)
        if (checkUren): # & (checkTabu):  #############
            
            a = list(set(crewlijst))
            try:
                a.remove(20)
            except:
                None

            if ((len(set(a) & set(crewA)) >= 1) | (len(crewA)==0)):
                lijstVrijeUren = [0]*16
                lijstVrijeUrenMorgen = [0]*16
                controle = True

                dagIndexKopie = dagIndex

                freq = 1
                while (freq <= int(taskdf.Frequency)) & (taskdf.Voltooid == False):
                    urenOver = 7 
                    urenOverMorgen = 7 
                    print("start van de while")        
                    if (taskdf.Eis != 'no') & (dagIndex == dagIndexKopie):
                        eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
                        lijstVrijeUren = [1]*int(eis["Moment Voltooid"]) + [0]*(16-int(eis["Moment Voltooid"]))
                    for i in crewlijst:
                        if (i == 20):
                            break
                        
                        if (float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]) >0):
                            urenOver = min(float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]), urenOver)
                            urenOverMorgen = min(float(dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[0,i+1]), urenOverMorgen)
                            print("uren: " + str(urenOver) + " " + str(urenOverMorgen))
                        else:
                            urenOver = 0
                            urenOverMorgen = 0
#                         print(str(urenOver)+ " " + str(urenOverMorgen) + " voor i: "+ str(i) +" in " +str(crewlijst) )
                            
                        if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(taskdf.Duration)):
                            controle = False
                            
                            
                            break
                            
                             
                        lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
                    duration = [0]*int(round(taskdf.Duration*2))
                    print("duration wordt gezet en controle: " + str(controle))
                    if (Hulpfuncties.contains(duration, lijstVrijeUren) != False) & controle:
#                         print("rij begin en eind")
                        rijbegin , rijeind = Hulpfuncties.contains(duration, lijstVrijeUren)

                        kamerGevonden = False
                        for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]:
                            if (duration == dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1].tolist()):
                                dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1] = taskdf.Task + " freq: " + str(freq)
                                kamerGevonden = True
                                controle = True
                                break
                            elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1])& (kamerGevonden == False)):
                                controle = False
                        if (controle == True):
                            print("controle is True")
                            for i in crewlijst:
                                if i == 20:
                                    break

                                dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task + " freq: " + str(freq) # + " " + str(taskdf.Mogelijkheden.dfMogelijkheden.iloc[crewlijst,0].iloc[0])
                                dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration)
#                             print("inplannen voor alle crew is gedaan")
                            if (freq == int(taskdf.Frequency)):
                                print("freq is max freq")
                                dfPrep = pd.DataFrame(dfAddPrep[dfAddPrep['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
                                dfPost = pd.DataFrame(dfAddPost[dfAddPost['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
#                                 print("crew toewijzen en zetten in dfSprint")
                                dfSprint.at[dfSprint.index[dfSprint['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                                geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                          prep = True)

                                geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                          prep = False)

                                if (geluktPost & geluktPrep):

                                    geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom,dfCrew=dfCrew, 
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                            prep = True, inplannen=True)
                                    geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                          prep = False, inplannen=True)

                                    dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Moment Voltooid'] = rijeind +1
                                    dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie
                                    dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Voltooid'] = True
                                    

                                    print(str(taskdf.Task) + " is ingepland, met prep en post")
                                    return [dfKalenderCrew, dfKalenderRoom, dfSprint, dfAddPrep, dfAddPost, dfCrew]
                            
                            print(str(taskdf.Task) + " freq: " + str(freq) + " crew: " +str(crewlijst) + "task voltooid: "+ str(taskdf.Voltooid))    
                            freq += 1
                        
                    else: # het inplannen over meerdere dagen

                        if ((urenOver > 0) & (urenOverMorgen > 0) & (dagIndexKopie != 9)): # controleren of mensen uren over hebben en of het niet de laatste dag van de week is.
                  
                            lijstVrijeEindUren = [0] * int(urenOver *2)

                            for i in crewlijst:
                                if (i == 20):
                                    break
                                eindUrenLijst = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[16- int(urenOver*2)+1:,i+1].tolist()
       
                                lijstVrijeEindUren = Hulpfuncties.CombinatieLijst(eindUrenLijst,lijstVrijeEindUren) # er zit een verschil in lengte
                            startMoment = get_last_non_zero_index(lijstVrijeEindUren)
                            durationEersteDag = len(lijstVrijeEindUren[startMoment:]) 

                            overigeDuration = taskdf.Duration*2 -durationEersteDag
                            lijstVrijeBeginUren = [0]* int(overigeDuration)
                            for i in crewlijst:
                                if (i == 20):
                                    break
                                beginUrenLijst = dfKalenderCrew.iloc[dagIndexKopie + 1,:].DagRooster.dfRooster.iloc[1:int(overigeDuration)+1,i+1].tolist()
                                lijstVrijeBeginUren = Hulpfuncties.CombinatieLijst(beginUrenLijst,lijstVrijeBeginUren)
                                print("begin: " + str(beginUrenLijst))
                                print("lijst: " + str(lijstVrijeBeginUren))
                            print("beginlijst: "+ str(lijstVrijeBeginUren) + "duration: " + str(overigeDuration))
                            past = index_of_first_nonzero(lijstVrijeBeginUren, overigeDuration)
                            print("test past: " +str(past))
                            if past:
                                print("past is true")
                                ## nu kunnen we deze inplannen.......
                                kamerGevonden = False

                                for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]:
                                    
                                    if ((16 - get_last_non_zero_index(dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[1:].tolist()) >= durationEersteDag) &
                                        (index_of_first_nonzero(dfKalenderRoom.iloc[dagIndexKopie+1,:].DagRooster.dfRooster[i].iloc[1:].tolist(),overigeDuration))):
                                        dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[16-durationEersteDag+1:] = taskdf.Task + " freq: " + str(freq)
                                        dfKalenderRoom.iloc[dagIndexKopie+1,:].DagRooster.dfRooster[i].iloc[1:int(overigeDuration)+1] = taskdf.Task + " freq: " + str(freq)
                                        print("kamer gevonden")
                                        kamerGevonden = True
                                        controle = True
                                        break
                                    elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1])& (kamerGevonden == False)):
                                        controle = False
                                        
                                # kamer gevonden, momenten gevonden
                                if (controle == True):

                                    for i in crewlijst:
                                        if i == 20:
                                            break
                                        print("inplannen voor: " + str(i))
                                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[16-durationEersteDag+1:,i+1] = taskdf.Task + " freq: " + str(freq) 
                                        dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[1:int(overigeDuration)+1,i+1] = taskdf.Task + " freq: " + str(freq) 
                                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - (durationEersteDag/2)
                                        dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[0,i+1] - (overigeDuration/2)
        #                             print("inplannen voor alle crew is gedaan")
                                    if (freq == int(taskdf.Frequency)):
                                        print("freq is max freq bij deels inplannen")
                                        dfPrep = pd.DataFrame(dfAddPrep[dfAddPrep['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
                                        dfPost = pd.DataFrame(dfAddPost[dfAddPost['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
        #                                 print("crew toewijzen en zetten in dfSprint")
                                        dfSprint.at[dfSprint.index[dfSprint['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                                        geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                                  dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                                  dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                                  prep = True)

                                        geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                                  dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                                  dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                                  prep = False)

                                        if (geluktPost & geluktPrep):

                                            geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                                  dfKalenderRoom=dfKalenderRoom,dfCrew=dfCrew, 
                                                                                                  dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                                    prep = True, inplannen=True)
                                            geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                                  dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                                  dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                                  prep = False, inplannen=True)

                                            dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Moment Voltooid'] = overigeDuration + 1
                                            dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie + 1
                                            dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Voltooid'] = True


                                            print(str(taskdf.Task) + " is ingepland, met prep en post")
                                            return [dfKalenderCrew, dfKalenderRoom, dfSprint, dfAddPrep, dfAddPost, dfCrew]

                                    print(str(taskdf.Task) + " freq: " + str(freq) + " crew: " +str(crewlijst) + "task voltooid: "+ str(taskdf.Voltooid))    
                                    freq += 1
                        
                                
                        dagIndexKopie+=1
                        print("dagIndex verhogen naar: " + str(dagIndexKopie))
                        controle = True
                        lijstVrijeUren = [0]*16
                        if (dagIndexKopie == 10):
                            
                            dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                            dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                            dfSprint = pickle.loads(pickle.dumps(dfTempSprint))
                            dfAddPrep = pickle.loads(pickle.dumps(dfTempPrep))
                            dfAddPost = pickle.loads(pickle.dumps(dfTempPost))
                            dfCrew = pickle.loads(pickle.dumps(dfCopyCrew))
                            break
    return [dfTempCrew, dfTempRoom, dfTempSprint, dfTempPrep, dfTempPost, dfCopyCrew]
    
    

In [32]:
dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
dfr = pickle.loads(pickle.dumps(dfKalenderRoom))
dfs = pickle.loads(pickle.dumps(dfSprint))
dfprep = pickle.loads(pickle.dumps(dfSprintAddprep))
dfpost = pickle.loads(pickle.dumps(dfSprintAddpost))
dfc = pickle.loads(pickle.dumps(dfCrew))

In [33]:
# filehandler = open('dfSprint1_eind.obj', 'rb') 
# dfSprint = pickle.load(filehandler)
# filehandler = open('dfKalenderCrew1.obj', 'rb') 
# dfKalenderCrew = pickle.load(filehandler)
# filehandler = open('dfKalenderRoom1.obj', 'rb') 
# dfKalenderRoom = pickle.load(filehandler)
# filehandler = open('dfSprintAddprep1.obj', 'rb') 
# dfSprintAddprep = pickle.load(filehandler)
# filehandler = open('dfSprintAddpost1.obj', 'rb') 
# dfSprintAddpost = pickle.load(filehandler)

In [34]:
# %%pixie_debugger

import warnings
warnings.filterwarnings("ignore")
# misschien bij een hogere iteratie een counter bijhouden voor de tasks die niet zijn ingepland.
iteratie = 0
while (dfSprint.Voltooid.sum()< len(dfSprint)) & (iteratie < 1):
    dfSprint = dfSprint.sort_values(['Voltooid', 'AantalMogelijkheden'], ascending= [True, False]).reset_index(drop=True)
    Eisen = dfSprint.Eis.unique().tolist()
    Eisen.remove('no')
    Eisen.append("S1C3T5B")
    Eisen.append("S1C1T1B")
    
    dfSprint = (dfSprint.loc[dfSprint['Task'].isin(Eisen)].append(dfSprint.loc[~dfSprint['Task'].isin(Eisen)])).reset_index(drop=True)
    rows = dfSprint.Task.tolist()
    
    dfKalenderCrew = pickle.loads(pickle.dumps(dfk))
    dfKalenderRoom = pickle.loads(pickle.dumps(dfr))
    dfSprintAddprep = pickle.loads(pickle.dumps(dfprep))
    dfSprintAddpost = pickle.loads(pickle.dumps(dfpost))
    dfCrew = pickle.loads(pickle.dumps(dfc))
    
    dfSprint = pickle.loads(pickle.dumps(dfs))
    dfSprint = dfSprint.set_index('Task')
    dfSprint = dfSprint.reindex(index= rows)
    dfSprint = dfSprint.reset_index()
    print("kopieen gemaakt, iteratie is: "+str(iteratie))
    for i in range(0,len(dfSprint)):
        task = dfSprint.iloc[i,:]
        print(task.Task)
        dagIndex = 5
#         dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
#         dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
#         dfTempSprint = pickle.loads(pickle.dumps(dfSprint)) 
        
        crewA = []
        if task.Eis != 'no':
            eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
            if eis.Voltooid == False: 
                print("eis inplannen")
                dfKalenderCrew, dfKalenderRoom, dfSprint, dfSprintAddprep, dfSprintAddpost, dfCrew = InplannenOpdracht(taskdf=eis,
                                                                   dagIndex=dagIndex, dfSprint=dfSprint,
                                                                   dfKalenderCrew=dfKalenderCrew, dfCrew=dfCrew,
                                                                   dfKalenderRoom=dfKalenderRoom,
                                                                   dfAddPrep=dfSprintAddprep, dfAddPost=dfSprintAddpost)
            eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
            dagIndex = int(eis['Dag Voltooid'])
            crewA = eis.Crew
        dfKalenderCrew, dfKalenderRoom, dfSprint, dfSprintAddprep, dfSprintAddpost, dfCrew = InplannenOpdracht(taskdf=task,
                                                               dagIndex=dagIndex, crewA=crewA,
                                                               dfKalenderCrew=dfKalenderCrew, dfCrew=dfCrew,
                                                               dfKalenderRoom=dfKalenderRoom,
                                                               dfAddPrep=dfSprintAddprep, dfAddPost=dfSprintAddpost,
                                                               dfSprint=dfSprint)
    iteratie += 1
    print(dfSprint.iloc[:,[0,20,21,22,23,27,28,29,30,31]])


kopieen gemaakt, iteratie is: 0
S1C1T1B
eis inplannen
start van de while
uren: 7.0 0.0
uren: 7.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T1A freq: 1 crew: [9, 0, 20, 20, 20, 20]task voltooid: False
start van de while
uren: 5.0 0.0
uren: 5.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T1A freq: 2 crew: [9, 0, 20, 20, 20, 20]task voltooid: False
start van de while
uren: 3.0 0.0
uren: 3.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T1A freq: 3 crew: [9, 0, 20, 20, 20, 20]task voltooid: False
start van de while
uren: 1.0 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 6
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
uren: 7.0 7.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
uren: 7.0 0.0
uren: 7.0 0.0
duration wordt gezet en controle: True
controle is True
freq is max freq
de crewList:   crew som
0    9  16
1   

controle is True
S1C1T1B freq: 2 crew: [12, 10, 4, 20, 20, 20]task voltooid: False
start van de while
uren: 1.0 7.0
duration wordt gezet en controle: False
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
beginlijst: [0, 0, 0, 0]duration: 4.0
test past: True
past is true
kamer gevonden
inplannen voor: 12
inplannen voor: 10
inplannen voor: 4
S1C1T1B freq: 3 crew: [12, 10, 4, 20, 20, 20]task voltooid: False
dagIndex verhogen naar: 9
start van de while
uren: 5.0 7.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
uren: 7.0 7.0
uren: 7.0 0.0
uren: 5.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T1B freq: 1 crew: [11, 9, 0, 20, 20, 20]task voltooid: False
start van de while
uren: 5.0 7.0
uren: 5.0 0.0
uren: 3.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T1B freq: 2 crew: [11, 9, 0, 20, 20, 20]task voltooid: False
start van de while
uren: 3.0 7

kamer gevonden
inplannen voor: 12
inplannen voor: 4
inplannen voor: 3
S1C1T1B freq: 3 crew: [12, 4, 3, 20, 20, 20]task voltooid: False
dagIndex verhogen naar: 9
start van de while
uren: 5.0 7.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
uren: 5.0 7.0
uren: 5.0 7.0
uren: 5.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T1B freq: 1 crew: [12, 6, 4, 20, 20, 20]task voltooid: False
start van de while
uren: 3.0 7.0
uren: 3.0 7.0
uren: 3.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T1B freq: 2 crew: [12, 6, 4, 20, 20, 20]task voltooid: False
start van de while
uren: 1.0 7.0
duration wordt gezet en controle: False
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
beginlijst: [0, 0, 0, 0]duration: 4.0
test past: True
past is true
kamer gevonden
inplannen voor: 12
inplannen voor: 6
inplannen voor: 4
S1C1T1B freq: 3 crew: [12, 6, 4, 20, 20, 20]t

start van de while
uren: 7.0 2.5
uren: 7.0 2.5
uren: 1.0 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 6
start van de while
uren: 2.5 0.0
uren: 2.5 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
uren: 7.0 0.0
uren: 7.0 0.0
uren: 5.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T4 freq: 1 crew: [10, 3, 0, 20, 20, 20]task voltooid: False
start van de while
uren: 5.0 0.0
uren: 5.0 0.0
uren: 3.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T4 freq: 2 crew: [10, 3, 0, 20, 20, 20]task voltooid: False
start van de while
uren: 3.0 0.0
uren: 3.0 0.0
uren: 1.0 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
uren: 7.0 2.5
uren: 7.0 2.5
uren: 1.0 0.0
duration wordt gezet en controle: False

uren: 7.0 0.0
uren: 3.0 0.0
uren: 3.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T4 freq: 5 crew: [10, 6, 3, 20, 20, 20]task voltooid: False
start van de while
uren: 5.0 0.0
uren: 1.0 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
uren: 7.0 2.5
uren: 7.0 2.5
duration wordt gezet en controle: False
dagIndex verhogen naar: 6
start van de while
uren: 2.5 0.0
uren: 2.5 0.0
uren: 2.5 0.0
duration wordt gezet en controle: True
controle is True
S1C1T4 freq: 1 crew: [10, 6, 5, 20, 20, 20]task voltooid: False
start van de while
uren: 0.5 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
uren: 7.0 0.0
uren: 3.0 0.0
uren: 3.0 0.0
duration wordt gezet en controle: True
controle is True
S1C1T4 freq: 2 crew: [10, 6, 5, 20, 20, 20]t

...

prep S1C2T1is daadwerkelijk ingepland
post S1C2T1 Niet ingepland of: False
S1C2T1 freq: 1 crew: [9, 8, 2, 1, 20, 20]task voltooid: False
start van de while
uren: 1.0 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 6
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
uren: 1.0 5.0
duration wordt gezet en controle: False
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
begin: ['S1C1T2 freq: 8', 'S1C1T2 freq: 8', 'S1C1T2 freq: 8', 'S1C1T2 freq: 8']
lijst: [1, 1, 1, 1]
begin: ['S1C1T4 freq: 5', 'S1C1T4 freq: 5', 'S1C1T4 freq: 5', 'S1C1T4 freq: 5']
lijst: [1, 1, 1, 1]
begin: ['S1C1T2 freq: 8', 'S1C1T2 freq: 8', 'S1C1T2 freq: 8', 'S1C1T2 freq: 8']
lijst: [1, 1, 1, 1]
beginlijst: [1, 1, 1, 1]duration: 4.0
test past: False
dagIndex verhogen naar: 8
start van de while
uren: 5.0 0.0
uren: 3.0 0.0
uren: 1.0 -2.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: Fals

start van de while
uren: 1.0 0.5
duration wordt gezet en controle: False
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
beginlijst: [1, 1, 1, 1]duration: 4.0
test past: False
dagIndex verhogen naar: 6
start van de while
uren: 0.5 1.0
duration wordt gezet en controle: False
begin: ['S1C1T4 freq: 4', 'S1C1T4 freq: 4', 'S1C1T4 freq: 4', 'S1C1T4 freq: 4']
lijst: [1, 1, 1, 1]
begin: ['S1C2T2A freq: 4', 'S1C2T2A freq: 4', 'S1C2T2A freq: 4', 'S1C2T2A freq: 4']
lijst: [1, 1, 1, 1]
begin: ['S1C2T3A freq: 1', 'S1C2T3A freq: 1', 'S1C2T3A freq: 1', 'S1C2T3A freq: 2']
lijst: [1, 1, 1, 1]
begin: ['S1C1T2 freq: 5', 'S1C1T2 freq: 5', 'S1C1

start van de while
uren: 1.0 0.5
duration wordt gezet en controle: False
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
beginlijst: [1, 1, 1, 1]duration: 4.0
test past: False
dagIndex verhogen naar: 6
start van de while
uren: 0.5 1.0
duration wordt gezet en controle: False
begin: ['S1C3T3 freq: 2', 'S1C3T3 freq: 2', 'S1C3T3 freq: 2', 'S1C3T3 freq: 2']
lijst: [1, 1, 1, 1]
begin: [0, 0, 0, 0]
lijst: [1, 1, 1, 1]
begin: ['S1C2T3A freq: 1', 'S1C2T3A freq: 1', 'S1C2T3A freq: 1', 'S1C2T3A freq: 2']
lijst: [1, 1, 1, 1]
begin: ['S1C1T2 freq: 5', 'S1C1T2 freq: 5', 'S1C1T2 freq: 5', 'S1C1T2 freq: 5']
lijst: [1, 1, 1, 1]
beginlijst: [

start van de while
uren: 1.0 5.0
duration wordt gezet en controle: False
begin: [0, 0, 0, 0]
lijst: [0, 0, 0, 0]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
begin: ['weekly stand-up', 'weekly stand-up', 'weekly stand-up', 'weekly stand-up']
lijst: [1, 1, 1, 1]
beginlijst: [1, 1, 1, 1]duration: 4.0
test past: False
dagIndex verhogen naar: 6
start van de while
uren: 5.0 3.0
uren: 0.5 0.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
uren: 3.0 1.0
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
uren: 1.0 5.0
duration wordt gezet en controle: False
begin: ['S1C1T3 freq: 8', 'S1C1T3 freq: 8', 'S1C1T3 freq: 8', 'S1C1T3 freq: 8']
lijst: [1, 1, 1, 1]
begin: ['S1C1T3 freq: 8', 'S1C1T3 freq: 8', 'S1C1T3 freq: 8', 'S1C1T3 freq: 8']
lijst: [1, 1, 1, 1]
begin: [0

KeyboardInterrupt: 

In [35]:
dfSprint.iloc[:,[0,20,21,22,23,25,27,28,29,30,31]]

,Task,MinReqCrew,MaxReqCrew,Frequency,Duration,Crew,AantalMogelijkheden,Voltooid,Eis,Moment Voltooid,Dag Voltooid
0,S1C1T1B,3.0,6.0,4.0,2.0,"[12, 6, 5, 20, 20, 20]",12.0,True,S1C1T1A,8,9
1,S1C1T1A,2.0,6.0,4.0,2.0,"[12, 0, 20, 20, 20, 20]",8.0,True,no,4,8
2,S1C2T2A,1.0,2.0,4.0,2.0,"[9, 20, 20, 20, 20, 20]",4.0,True,no,4,7
3,S1C2T3A,1.0,2.0,2.0,1.5,"[4, 20, 20, 20, 20, 20]",2.0,True,no,6,7
4,S1C3T5A,1.0,2.0,2.0,1.5,"[5, 20, 20, 20, 20, 20]",2.0,True,no,6,5
5,S1C3T5B,1.0,2.0,2.0,2.0,"[5, 20, 20, 20, 20, 20]",2.0,True,S1C3T5A,14,5
6,S1C3T2A,1.0,6.0,1.0,2.0,"[11, 20, 20, 20, 20, 20]",1.0,True,no,4,5
7,S1C1T4,3.0,6.0,8.0,2.0,"[11, 10, 3, 20, 20, 20]",24.0,True,no,5,9
8,S1C1T2,2.0,6.0,8.0,2.0,"[8, 2, 20, 20, 20, 20]",16.0,True,no,5,8
9,S1C1T3,2.0,6.0,8.0,2.0,"[13, 7, 20, 20, 20, 20]",16.0,True,no,5,9


In [ ]:
dfSprintAddpost.iloc[:,[0,20,21,22,23,27,28,31,32,33,34]]

In [ ]:
dfSprintAddprep.iloc[:,[0,20,21,22,23,27,28,31,32,33,34]]

In [ ]:
Controleren.EvaluateProgram(dfSprint,dfk,dfKalenderCrew)

In [ ]:
dfKalenderCrew.iloc[1,:].DagRooster.dfRooster

In [ ]:
dfKalenderCrew.iloc[45,:].DagRooster.dfRooster

In [36]:
dfKalenderCrew.iloc[7,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston,Extra 1,Extra 2,Extra 3
0,urenTeGaan,0,0,1,-2,0,1,0,0,1,1,-2,1,1,3,3,7
1,09:00:00,0,0,S1C1T2 freq: 5,S1C1T4 freq: 4,S1C2T3A freq: 1,S1C3T3 freq: 2,0,0,S1C1T2 freq: 5,S1C2T2A freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C3T3 freq: 2,0,0,0
2,09:30:00,0,0,S1C1T2 freq: 5,S1C1T4 freq: 4,S1C2T3A freq: 1,S1C3T3 freq: 2,0,0,S1C1T2 freq: 5,S1C2T2A freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C3T3 freq: 2,0,0,0
3,10:00:00,0,0,S1C1T2 freq: 5,S1C1T4 freq: 4,S1C2T3A freq: 1,S1C3T3 freq: 2,0,0,S1C1T2 freq: 5,S1C2T2A freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C3T3 freq: 2,0,0,0
4,10:30:00,0,0,S1C1T2 freq: 5,S1C1T4 freq: 4,S1C2T3A freq: 2,S1C3T3 freq: 2,0,0,S1C1T2 freq: 5,S1C2T2A freq: 4,S1C1T4 freq: 4,S1C1T4 freq: 4,S1C3T3 freq: 2,0,0,0
5,11:00:00,0,0,S1C1T2 freq: 6,0,S1C2T3A freq: 2,S1C3T3 freq: 3,0,0,S1C1T2 freq: 6,S1C2T2B freq: 1,0,S1C2T2B freq: 1,S1C3T3 freq: 3,S1C2T2B freq: 1,0,0
6,11:30:00,0,0,S1C1T2 freq: 6,0,S1C2T3A freq: 2,S1C3T3 freq: 3,0,0,S1C1T2 freq: 6,S1C2T2B freq: 1,0,S1C2T2B freq: 1,S1C3T3 freq: 3,S1C2T2B freq: 1,0,0
7,12:30:00,0,0,S1C1T2 freq: 6,0,S1C2T3B freq: 1,S1C3T3 freq: 3,0,0,S1C1T2 freq: 6,S1C2T2B freq: 1,0,S1C2T2B freq: 1,S1C3T3 freq: 3,S1C2T2B freq: 1,S1C2T3B freq: 1,0
8,13:00:00,0,0,S1C1T2 freq: 6,0,S1C2T3B freq: 1,S1C3T3 freq: 3,0,0,S1C1T2 freq: 6,S1C2T2B freq: 1,0,S1C2T2B freq: 1,S1C3T3 freq: 3,S1C2T2B freq: 1,S1C2T3B freq: 1,0
9,13:30:00,0,0,S1C1T2 freq: 7,0,S1C2T3B freq: 1,S1C3T3 freq: 4,0,0,S1C1T2 freq: 7,S1C2T2B freq: 2,0,S1C2T2B freq: 2,S1C3T3 freq: 4,S1C2T2B freq: 2,S1C2T3B freq: 1,0


In [ ]:
dfKalenderCrew.iloc[8,:].DagRooster.dfRooster

In [ ]:
dfKalenderCrew.iloc[9,:].DagRooster.dfRooster

In [ ]:
get_last_non_zero_index(dfKalenderRoom.iloc[5,:].DagRooster.dfRooster['technical A'].iloc[1:].tolist())

In [ ]:
dfKalenderRoom.iloc[5,:].DagRooster.dfRooster

In [ ]:
dfSprint.iloc[0,:].Mogelijkheden.dfMogelijkheden.iloc[0,0]

!TODO 99% betrouwbaarheidsinterval

Het bedrijf werkt in sprints. Om precies te zijn: 4 sprints van elk 10 weken. De weken
2 t/m 8 van elke sprint zijn de zogeheten projectweken

In [ ]:
# # het kopieren van de roosters. Deze zijn voor dit gedeelte gelijk in week 2 tot en met 8. 


for i in range(1,8):
    for j in range(5,10):
        dfKalenderRoom.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderRoom.iloc[j,:].DagRooster.dfRooster))
        dfKalenderCrew.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderCrew.iloc[j,:].DagRooster.dfRooster))
    

!TODO de mensen die de prep uitvoeren, moeten ook de projecttasks en post doen.

Wanneer kan er met de prep check-up worden begonnen en wanneer kan de check-up worden gedaan?

In [ ]:
dfCrew

In [ ]:
# # back-up maken van de verschillende dataframes zodat deze kunnen worden terug gezet.

# dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
# dfs = pickle.loads(pickle.dumps(dfSprint))
# dfr = pickle.loads(pickle.dumps(dfKalenderRoom))


!TODO kiezen van de crew director

In [ ]:
# for i in range(0,len(dfSprintAddprep)):
#     task = dfSprintAddprep.iloc[i,:]

#     dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
#     dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
#     print(task.Task)
#     dfKalenderCrew, dfKalenderRoom = InplannenAdd(task, dfKalenderCrew, dfKalenderRoom, 
#                                                       dfSprint=dfSprint, dfAdd= dfSprintAddprep)

In [ ]:
# for i in range(0,len(dfSprintAddpost)):
#     task = dfSprintAddpost.iloc[i,:]

#     dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
#     dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
#     print(task.Task)
#     dfKalenderCrew, dfKalenderRoom = InplannenAdd(task, dfKalenderCrew, dfKalenderRoom, 
#                                                       dfSprint=dfSprint, dfAdd= dfSprintAddpost)

In [ ]:
# filehandler = open('dfSprint1_eind.obj', 'wb') 
# pickle.dump(dfSprint, filehandler)

# filehandler = open('dfKalenderCrew1.obj', 'wb') 
# pickle.dump(dfKalenderCrew, filehandler)

# filehandler = open('dfKalenderRoom1.obj', 'wb') 
# pickle.dump(dfKalenderRoom, filehandler)

# filehandler = open('dfSprintAddprep1.obj', 'wb') 
# pickle.dump(dfSprintAddprep, filehandler)

# filehandler = open('dfSprintAddpost1.obj', 'wb') 
# pickle.dump(dfSprintAddpost, filehandler)

# filehandler = open('dfCrew1.obj', 'wb') 
# pickle.dump(dfCrew, filehandler)

# filehandler.close()